Init

In [1]:
import robotic as ry
import time
with open("logger.txt","w") as f:
    f.write("")
try:
    del C
    del bot
except:
    pass

C = ry.Config()
# C.watchFile("throwing1.g")
C.addFile("throwing_bare.g")
qHome = C.getJointState()
C.view()
#time.sleep(30)
C.getFrame('cargo').unLink()
C.view()

0

Grasping

In [2]:
def pre_grasp_komo(C, gripper_name, grasp_frame_name, q0, qHome):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) 
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)

    komo.addObjective([1., 3.], ry.FS.positionRel, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0, 0, 0])  

    # komo.addObjective([1., 3.], ry.FS.scalarProductXX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 
    komo.addObjective([1., 3.], ry.FS.scalarProductYX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e2], [1]) 
    # komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 

    komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [grasp_frame_name, gripper_name], ry.OT.eq, [1e1], [1]) 


    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret)
    return komo
def create_grasp_komo(C, gripper_name, grasp_frame_name, q0, qHome):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) 
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)

    komo.addObjective([1., 3.], ry.FS.positionRel, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0, 0, 0])  

    # komo.addObjective([1., 3.], ry.FS.scalarProductXX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 
    komo.addObjective([1., 3.], ry.FS.scalarProductYX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e2], [1]) 
    # komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 

    komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [grasp_frame_name, gripper_name], ry.OT.eq, [1e1], [1]) 


    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret)
    return komo
q0 = C.getJointState()
komo0 = pre_grasp_komo(C, 'l_gripper', 'cargo', q0, qHome)
path0 = komo0.getPath()

komo = create_grasp_komo(C, 'l_gripper', 'cargo', q0, qHome)
path = komo.getPath()
bot = ry.BotOp(C, useRealRobot=False)
bot.home(C)
# print(path.shape)
bot.move(path0, [1.])
bot.move(path, [3.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

#bot.gripperCloseGrasp(ry._left, 'cargo',speed=1000000000000,width=0.04)
bot.gripperClose(ry._left,width=0.06) 
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

#C.getFrame('cargo').unLink()
C.attach('l_gripper', 'cargo')

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):16717.2 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    11433.1  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    6774.05  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    3582.19  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    2006.91  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    1020.95  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):    404.453  ACCEPT
--newton-- it:   7  |Delta|:        0.2  alpha:          1  evals:   8  f(y):    82.9403  ACCEPT
--newton-- it:   8  |Delta|:   0.178216  alpha:          1  evals:   9  f(y):    6.26522  ACCEPT
--newton-- it:   9  |Delta|:  0.0909678  alpha:          1  evals:  10  f(y

Now for the fun part. Create 2 points initial point and throwing point

In [3]:
import numpy as np
import json
C.addFrame("initial_point").setPosition([0.3,2,0.4]).setShape(ry.ST.marker,[.3]).setColor([0,1,0])
C.addFrame("release_point").setPosition([0.6,2,0.4]).setShape(ry.ST.marker,[.3]).setColor([1,0,0])
release_velocity = np.array([2,0,1])
C.view()
def throw_init():
    q0 = C.getJointState()
    komo = ry.KOMO(C, 1, 1, 1, True)
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)
    komo = ry.KOMO(C, 1, 1, 0, True)
    komo.addObjective([], ry.FS.positionDiff, ["l_gripper", "initial_point"], ry.OT.eq, [1e1], [0, 0, 0])
    komo.addObjective([], ry.FS.scalarProductXZ, ["initial_point","l_gripper"], ry.OT.eq, [1e1], [-1])
    ret2 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(komo.report())
    print(ret2)
    return komo

def throw_komo():
    q0 = C.getJointState()
    komo = ry.KOMO(C, 1, 1, 2, True)
    komo.addControlObjective([], 1, 1e0)
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)
    komo.addObjective([1.], ry.FS.positionDiff, ["l_gripper", "release_point"], ry.OT.eq, [1e1], [0, 0, 0],order=0)
    #komo.addObjective([1.], ry.FS.positionDiff, ["l_gripper", "release_point"], ry.OT.sos, [1e-2])
    #komo.addObjective([1.], ry.FS.scalarProductYX, ["release_point", "l_gripper"], ry.OT.eq, [1e1], [-1],order=0)
    komo.addObjective([1.], ry.FS.position, ["l_gripper"], ry.OT.eq, [1e1], [5],order=1)
    #komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [5], order=1)
    #komo.addObjective([1.], ry.FS.position, ["l_gripper"], ry.OT.eq, [1e1], [1,0,0],order=2)
    #print(C.eval(ry.FS.position, frames=['l_gripper',"release_point"], order=1))
    ret2 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(json.dumps(komo.report(),indent=2))
    print(ret2)
    return komo

komo_init = throw_init()
komo_throw = throw_komo()
print(komo_throw.getPathTau())
rel_frame = C.getFrame("release_point")
grip_frame = C.getFrame("l_gripper")
path_init = komo_init.getPath()
path_throw = komo_throw.getPath()
bot.move(path_init,[3.])
while bot.getTimeToEnd()>0:
    bot.sync(C,.1)
print(f"After init position:{bot.getTimeToEnd()}")
bot.move(path_throw,[1.])
isApplied = False
while bot.getTimeToEnd() > 0:
    #print(C.eval(ry.FS.position,["cargo","base"],target=release_velocity,order=1))
    if  np.linalg.norm(rel_frame.getPosition()-grip_frame.getPosition())<0.05 and not isApplied:
        print("Entered")
        bot.gripperMove(ry._left,speed=0.9)
        isApplied = True
    bot.sync(C, .1)

time.sleep(5)
ry.Skeleton()

{'o1': {'name': 'F_ScalarProduct', 'frames': ['initial_point', 'l_gripper'], 'err': 0.19116691941572173, 'order': 0.0, 'type': 'eq', 'scale': [10.0], 'target': [-1.0], 'times': [], 'slices': [0.0, 0.0]}, 'o0': {'name': 'F_PositionDiff', 'frames': ['l_gripper', 'initial_point'], 'err': 0.35661311185543276, 'order': 0.0, 'type': 'eq', 'scale': [10.0], 'target': [0.0, 0.0, 0.0], 'times': [], 'slices': [0.0, 0.0]}, 'totals': {'sos': 0.0, 'ineq': 0.0, 'eq': 0.5477800312711545, 'f': 0.0}}====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):123.688 alpha:1 beta:1
--newton-- it:   1
{ time: 0.006299, evals: 150, done: 1, feasible: 0, sos: 0, f: 0, ineq: 0, eq: 0.54778 }
  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    104.025  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    84.1287  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):   

In [4]:
del C
del bot

-- bot.cpp:~BotOp:135(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:57(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation
